In [1]:
import numpy as np
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'AffectNet_train_set/seven/'
validation = ALL_DATA_DIR + 'AffectNet_val_set/seven/'

In [3]:
batch_size = 32 
img_height = 224
img_width = 224 

In [4]:
training_s = tf.keras.utils.image_dataset_from_directory(training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)
validation_s = tf.keras.utils.image_dataset_from_directory(validation, labels = 'inferred', 
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 283901 files belonging to 7 classes.
Found 3500 files belonging to 7 classes.


In [5]:
base_model = keras.applications.EfficientNetB4(input_shape=(224,224,3), include_top = False, weights="imagenet")

In [6]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [7]:
for layer in base_model.layers:
    layer.trainable = False

In [8]:
model.compile(optimizer="Adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(training_s, validation_data=validation_s, epochs=5)
model.save('0919_Affect_EfB4_frozen.h5')

Epoch 1/5
8872/8872 [==============================] - 1311s 146ms/step - loss: 1.0247 - accuracy: 0.6322 - val_loss: 2.0572 - val_accuracy: 0.3254
Epoch 2/5
8872/8872 [==============================] - 1320s 149ms/step - loss: 0.9781 - accuracy: 0.6481 - val_loss: 1.9505 - val_accuracy: 0.3449
Epoch 3/5
8872/8872 [==============================] - 1321s 149ms/step - loss: 0.9670 - accuracy: 0.6517 - val_loss: 2.0263 - val_accuracy: 0.3351
Epoch 4/5
8872/8872 [==============================] - 1309s 148ms/step - loss: 0.9620 - accuracy: 0.6543 - val_loss: 1.9880 - val_accuracy: 0.3557
Epoch 5/5
8872/8872 [==============================] - 1310s 148ms/step - loss: 0.9583 - accuracy: 0.6544 - val_loss: 2.0232 - val_accuracy: 0.3400


In [10]:
for layer in base_model.layers:
    layer.trainable = True

In [11]:
opt = keras.optimizers.Adam(learning_rate=1e-4)

In [12]:
es = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [13]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_s, validation_data=validation_s, epochs=20, callbacks=[es])

Epoch 1/20
8872/8872 [==============================] - 6506s 731ms/step - loss: 0.6909 - accuracy: 0.7495 - val_loss: 1.3321 - val_accuracy: 0.5380
Epoch 2/20
8872/8872 [==============================] - 6486s 731ms/step - loss: 0.5647 - accuracy: 0.7922 - val_loss: 1.2522 - val_accuracy: 0.5740
Epoch 3/20
8872/8872 [==============================] - 6476s 730ms/step - loss: 0.4738 - accuracy: 0.8236 - val_loss: 1.4375 - val_accuracy: 0.5609
Epoch 4/20
8872/8872 [==============================] - 6470s 729ms/step - loss: 0.3708 - accuracy: 0.8600 - val_loss: 1.5825 - val_accuracy: 0.5751
Epoch 5/20
8872/8872 [==============================] - 6477s 730ms/step - loss: 0.2842 - accuracy: 0.8919 - val_loss: 1.8728 - val_accuracy: 0.5669
Epoch 6/20
8872/8872 [==============================] - 6479s 730ms/step - loss: 0.2285 - accuracy: 0.9143 - val_loss: 2.0817 - val_accuracy: 0.5617
Epoch 7/20
8872/8872 [==============================] - 6478s 730ms/step - loss: 0.1897 - accuracy: 0.9289

In [14]:
model.save("0919_Affect_B4_eval.h5")